In [20]:
import subprocess
import os

path = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/checkpoints/'
methods = ['2d_unet_neuron_z32', '3d_unet_neuron_z32', '3d_linknet_neuron']

real_path = path+methods[1]+'/test_full/real_B_project/'
fake_path = path+methods[1]+'/test_full/fake_B_project/'

total = 0
for _,(real,fake) in enumerate(zip(os.listdir(real_path), os.listdir(fake_path))):
#     print(real,fake)
#     print(real_path+real, fake_path+fake)
    cmd = ['pyssim', real_path+real, fake_path+fake]
    output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
    print(output.decode("utf-8"))
    total+=float(output.decode("utf-8"))
print(total/5)

0.5011055

0.4318894

0.575716

0.5300824

0.6326412

0.5342869


In [ ]:
0.6821576

0.7400352

0.5422718

0.655771

0.5273365


0.5011055

0.4318894

0.575716

0.5300824

0.6326412

In [41]:
def loadtiff3d(filepath):
    """Load a tiff file into 3D numpy array"""

    import tifffile as tiff
    a = tiff.imread(filepath)

    stack = []
    for sample in a:
        stack.append(np.rot90(np.fliplr(np.flipud(sample))))
    out = np.dstack(stack)

    return out

def writetiff3d(filepath, block):
    import tifffile as tiff

    try:
        os.remove(filepath)
    except OSError:
        pass

    with tiff.TiffWriter(filepath, bigtiff=False) as tif:
        for z in range(block.shape[2]):
            saved_block = np.rot90(block[:, :, z])
            tif.save(saved_block.astype('uint8'), compress=0)

In [ ]:
(207, 175, 151)
0.949205702591625
(190, 241, 127)
0.9298947389583114
(147, 209, 207)
0.8786159546436956
(153, 203, 116)
0.8403984850399471
(65, 105, 95)
0.7149634649807374
average: 0.8626156692428634

In [77]:
from skimage import measure
import cv2
import numpy as np
import math

path = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/checkpoints/'
methods = ['2d_unet_neuron_z32', '3d_unet_neuron_z32', '3d_linknet_neuron']

real_path = path+methods[1]+'/test_full/real_B/'
fake_path = path+methods[1]+'/test_full/fake_B/'

total_ssim = 0
total_mse = 0
total_psnr_1 = 0
total_psnr_2 = 0
for _,(real,fake) in enumerate(zip(os.listdir(real_path), os.listdir(fake_path))):
    a = loadtiff3d(real_path+real)
    b = loadtiff3d(fake_path+fake)
    
    s = measure.compare_ssim(a, b)
    m = mse(a,b)
    psnr_1 = psnr1(a,b)
    psnr_2 = psnr2(a,b)
    total_ssim += s
    total_mse += m
    total_psnr_1 += psnr_1
    total_psnr_2 += psnr_2
#     print(a.shape)
    print(s)
    print(m)
    
    
print('average SSIM:', total_ssim/5)
print('average mse:', total_mse/5)
print('average psnr1:', total_psnr_1/5)
print('average psnr2:', total_psnr_2/5)

0.9536027670736983
36.8797522840598
0.9529356751639284
29.883761368658355
0.925281883330857
35.04958424670749
0.8269231907544411
42.290504667978965
0.6710535649510824
190.48211760169656
average SSIM: 0.8659594162548014
average mse: 66.91714403382022
average psnr1: 31.144797200405254
average psnr2: 31.144797200405254


In [ ]:
U-net
0.9536027670736983
36.8797522840598
0.9529356751639284
29.883761368658355
0.925281883330857
35.04958424670749
0.8269231907544411
42.290504667978965
0.6710535649510824
190.48211760169656
average SSIM: 0.8659594162548014
average mse: 66.91714403382022
average psnr1: 31.144797200405254
average psnr2: 31.144797200405254
    
Link-net
0.949205702591625
38.187937239201275
0.9298947389583114
34.44531591500397
0.8786159546436956
44.692158591472094
0.8403984850399471
46.61583321398317
0.7149634649807374
212.76616464237517
average SSIM: 0.8626156692428634
average mse: 75.34148192040713
average psnr1: 30.599356169078483
average psnr2: 30.599356169078483

In [65]:
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * imageA.shape[2])
    
    return err

In [72]:
def psnr1(img1, img2):
   mse = np.mean((img1.astype("float") - img2.astype("float")) ** 2 )
   if mse < 1.0e-10:
      return 100
   return 10 * math.log10(255.0**2/mse)

def psnr2(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [68]:
path = '/media/jacktang/Work/USYD/Research/Deep_Learning/GAN/pytorch-CycleGAN-and-pix2pix/checkpoints/'
methods = ['2d_unet_neuron_z32', '3d_unet_neuron_z32', '3d_linknet_neuron']

real_path = path+methods[1]+'/test_full/real_B/'
fake_path = path+methods[1]+'/test_full/fake_B/'

real = loadtiff3d(real_path+'22_real_B.tif')
fake = loadtiff3d(fake_path+'22_fake_B.tif')
empty = np.zeros_like(fake)
fill = np.ones_like(fake)
# print(np.unique(a))
# print(np.unique(b))
print(measure.compare_ssim(fake, real))
print(measure.compare_ssim(empty, real))
print(measure.compare_ssim(fill, real))
print(mse(fake, real))
print(mse(empty, real))
print(mse(fill, real))

0.925281883330857
0.9563718927354815
0.8409747123412363
35.04958424670749
37.64144110826033
37.848101022994776
